# *Unsupervised learning: Latent Dirichlet allocation (LDA) topic modeling*

In [ ]:
## Install Python package for LDA
# http://pythonhosted.org/lda/getting_started.html

!pip3 install lda

In [ ]:
## Importing basic packages

import os
import numpy as np

In [ ]:
os.chdir('/sharedfolder/')

!wget https://github.com/pcda17/pcda17.github.io/raw/master/week/10/nyt_articles_11-9-2017.zip

!unzip nyt_articles_11-9-2017.zip

os.chdir('/sharedfolder/nyt_articles_11-9-2017/')

In [ ]:
document_list = []

for filename in [item for item in os.listdir('./') if '.txt' in item]:
    text_data = open(filename).read()
    document_list.append(text_data)

In [ ]:
## Importing NLTK stop words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

 
stop_words = stopwords.words('english') + ["'s", "'re", '”', '“', '’', '—'] + list(string.punctuation)

string.punctuation

In [ ]:
## Tokenizing and removing stop words from our list of documents

documents_filtered = []

for document in document_list:
    token_list = word_tokenize(document.lower())
    tokens_filtered = [item for item in token_list if (item not in stop_words)]
    documents_filtered.append(' '.join(tokens_filtered))

In [ ]:
## Viewing a preprocessed document

documents_filtered[30]

In [ ]:
## Vectorizing preprocessed essays

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents_filtered) 

In [ ]:
## Creating a vocabulary list corresponding to the vectors we created above

vocabulary = vectorizer.get_feature_names()

vocabulary[1140:1160]

In [ ]:
## Initializing an LDA model: 10 topics and 1500 iterations

import lda

model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)

In [ ]:
## Fitting the model using our list of vectors

model.fit(X)

In [ ]:
## Viewing the top 50 words in each 'topic'

topic_word = model.topic_word_

n_top_words = 100

for i, topic_distribution in enumerate(topic_word):
    topic_words = np.array(vocabulary)[np.argsort(topic_distribution)][:-(n_top_words+1):-1]
    print('Topic ' + str(i) + ':')
    print(' '.join(topic_words))
    print()

### ▷Assignment

    Modify the code above: Apply a stemming step to each word before vectorizing the text.
    See example stemming code in the following cell.

In [ ]:
## Stemming example

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

print(stemmer.stem('nature'))

print(stemmer.stem('natural'))

print(stemmer.stem('naturalism'))

# *Supervised learning: Naive Bayes classification*

In [ ]:
## Download sample text corpora from GitHub, then unzip.

os.chdir('/sharedfolder/')

## Uncomment the lines below if you need to re-download test corpora we used last week.

#!wget -N https://github.com/pcda17/pcda17.github.io/blob/master/week/8/Sample_corpora.zip?raw=true -O Sample_corpora.zip
#!unzip -o Sample_corpora.zip

In [ ]:
os.chdir('/sharedfolder/Sample_corpora')

os.listdir('./')

In [ ]:
## Loading Melville novels

os.chdir('/sharedfolder/Sample_corpora/Herman_Melville/')

melville_texts = []

for filename in os.listdir('./'):
    text_data = open(filename).read().replace('\n', ' ')
    melville_texts.append(text_data)

print(len(melville_texts))

In [ ]:
## Loading Austen novels

os.chdir('/sharedfolder/Sample_corpora/Jane_Austen/')

austen_texts = []

for filename in os.listdir('./'):
    text_data = open(filename).read().replace('\n', ' ')
    austen_texts.append(text_data)

print(len(austen_texts))

In [ ]:
## Removing the last novel from each list so we can use it to test our classifier

melville_train_texts = melville_texts[:-1]
austen_train_texts = austen_texts[:-1]

melville_test_text = melville_texts[-1]
austen_test_text = austen_texts[-1]

In [ ]:
## Creating a master list of Melville sentences

from nltk.tokenize import sent_tokenize

melville_combined_texts = ' '.join(melville_train_texts)

melville_sentences = sent_tokenize(melville_combined_texts)

print(len(melville_sentences))

melville_sentences[9999]

In [ ]:
## Extracting 2000 Melville sentences at random for use as a training set

import random

melville_train_sentences = random.sample(melville_sentences, 2000)

In [ ]:
## Creating a list of Melville sentences for our test set

melville_test_sentences = sent_tokenize(melville_test_text)

print(len(melville_test_sentences))

melville_test_sentences[997]

In [ ]:
## Creating a master list of Austen sentences

austen_combined_texts = ' '.join(austen_train_texts)

austen_sentences = sent_tokenize(austen_combined_texts)

print(len(austen_sentences))

austen_sentences[8979]

In [ ]:
## Extracting 2000 Austen sentences at random for use as a training set

austen_train_sentences = random.sample(austen_sentences, 2000)

In [ ]:
## Creating a list of Austen sentences for our test set

austen_test_sentences = sent_tokenize(austen_test_text)

print(len(austen_test_sentences))

austen_test_sentences[1000]

In [ ]:
## Combing training data
combined_texts = melville_train_sentences + austen_train_sentences

## Creating list of associated class values: 
## 0 for Melville, 1 for Austen
y = [0]*len(melville_train_sentences) + [1]*len(austen_train_sentences)

In [ ]:
## Creating vectorized training set using our combined sentence list

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(combined_texts).toarray()

X.shape

In [ ]:
## Training a multinomial naive Bayes classifier
## X is a combined list of Melville and Austen sentences (2000 sentences from each)
## y is a list of classes (0 or 1)

from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(X, y)

In [ ]:
## Classifying 5 sentences in our Austen test set
# Recall that 0 means Melville & 1 means Austen

from pprint import pprint

input_sentences = austen_test_sentences[3000:3005]

input_vector = vectorizer.transform(input_sentences)   ## Converting a list of string to the same
                                                       ## vector format we used for our training set.
pprint(austen_test_sentences[3000:3005])

classifier.predict(input_vector)

In [ ]:
## Classifying 5 sentences in our Melville test set

input_sentences = melville_test_sentences[3000:3005]

input_vector = vectorizer.transform(input_sentences)

pprint(melville_test_sentences[3000:3005])

classifier.predict(input_vector)

### ▷Assignment

    Write a script that prints Austen-like sentences written 
    by Melville, and Melville-like sentences written by Austen.